In [91]:
import numpy as np
import pandas as pd

In [92]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline,make_pipeline# for pipline
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectKBest,chi2

In [93]:
df=pd.read_csv("train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Let's Plan

In [94]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [95]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [96]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [97]:
X_train,X_test,y_train,y_test=train_test_split(df.drop(columns=['Survived']),
                                               df["Survived"],
                                              train_size=0.2,
                                              random_state=42)

In [98]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
761,3,male,41.0,0,0,7.1250,S
645,1,male,48.0,1,0,76.7292,C
754,2,female,48.0,1,2,65.0000,S
556,1,female,48.0,1,0,39.6000,C
850,3,male,4.0,4,2,31.2750,S


In [99]:
y_train.sample(5)

701    1
14     0
574    0
719    0
166    1
Name: Survived, dtype: int64

In [100]:
# imputation transformer
# instead of using column name we use index value of column otherwise code will not work properly
#passthough means take only necessary column and other skip
trf1=ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')

In [101]:
#onehotencoding
trf2=ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse=False,handle_unknown="ignore"),[1,6])
],remainder='passthrough')

In [102]:
# scaling
#apply in all column from 0 to 10 all****
trf3=ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])

In [103]:
# Feature Scelection
trf4=SelectKBest(score_func=chi2,k=8)

In [104]:
# trainn thmodels
trf5=DecisionTreeClassifier()

# Create Pipline

In [105]:
pipe=Pipeline([
    ("trf1",trf1),
    ("trf2",trf2),
    ("trf3",trf3),
    ("trf4",trf4),
    ("trf5",trf5)
])

# Pipeline Vs make_pipeline
Pipeline requires naming of steps, make_pipeline does not.

(Same applies to ColumnTransformer vs make_column_transformer)

In [106]:
# Alternate Syntex
pipe=make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [107]:
#train
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x0000024645968DC0>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

# Explore the Pipeline

In [108]:
pipe.named_steps

{'columntransformer-1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'columntransformer-2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse=False),
                                  [1, 6])]),
 'columntransformer-3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'selectkbest': SelectKBest(k=8, score_func=<function chi2 at 0x0000024645968DC0>),
 'decisiontreeclassifier': DecisionTreeClassifier()}

In [109]:
#Display Pipline
from sklearn import set_config
set_config(display="diagram")    

In [110]:
# Predict
y_pred=pipe.predict(X_test)

In [111]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6086956521739131

# Cross Validation Using Pipline

In [112]:
#cross validation using cross val_score
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,X_train,y_train,cv=5,scoring='accuracy').mean()

0.6123809523809525

# GridSearch Using Pipline

In [116]:
# export 
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))